In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.pylab as plta
import matplotlib.ticker as ticker

import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error as MSE

plt.style.use('seaborn')

In [2]:
df = pd.read_csv (r'C:\אריאל\שנה א\סמסטר א\מבוא למדעי הנתונים\עבודת הגמר\Data_Science_intro_project\Classification\dataclassification_converted_to_english.csv', index_col = False)

In [3]:
df.head()

,product_name,code,category_name,Vitamin_B1,Vitamin_B6,Dietary_fiber,Folic_acid,energy,Proteins,Carbohydrates,...,K_Potassium,humidity,chromium,Molybdenum,Unsaturated,Acidic_acid,Linoleic_acid,DHA,EPA,Chlorophyll
0,clementine,P_964560,fruits,0.09,0.08,1.7,24.0,47.0,9.0,12.0,...,0,0,0,0,0,0,0,0,0,0.0
1,Gilded_apple,P_963136,fruits,0.00,0.00,2.4,0.0,52.0,0.3,13.8,...,0,0,0,0,0,0,0,0,0,0.0
2,lemon,P_964492,fruits,0.00,0.08,2.8,0.0,29.0,1.1,9.3,...,0,0,0,0,0,0,0,0,0,0.0
3,Orange_in_the_juice_network,P_964348,fruits,0.09,0.06,2.5,39.0,49.0,1.0,11.9,...,0,0,0,0,0,0,0,0,0,0.0
4,Avocado,P_964980,fruits,0.07,0.30,6.7,81.0,160.0,2.0,8.5,...,0,0,0,0,0,0,0,0,0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9581 entries, 0 to 9580
Data columns (total 100 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   product_name                         9581 non-null   object 
 1   code                                 9581 non-null   object 
 2   category_name                        9581 non-null   object 
 3   Vitamin_B1                           9581 non-null   float64
 4   Vitamin_B6                           9581 non-null   float64
 5   Dietary_fiber                        9581 non-null   float64
 6   Folic_acid                           9581 non-null   float64
 7   energy                               9580 non-null   float64
 8   Proteins                             9580 non-null   float64
 9   Carbohydrates                        9566 non-null   float64
 10  Fats                                 9581 non-null   object 
 11  sodium                       

In [5]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state=42)

In [6]:
prod = train_set.copy()

In [7]:
prod.replace('(^\s+|\s+$)', '', regex=True, inplace=True)
prod = prod.set_index(['product_name', 'code'])

In [8]:
prod

,,category_name,Vitamin_B1,Vitamin_B6,Dietary_fiber,Folic_acid,energy,Proteins,Carbohydrates,Fats,sodium,...,K_Potassium,humidity,chromium,Molybdenum,Unsaturated,Acidic_acid,Linoleic_acid,DHA,EPA,Chlorophyll
product_name,code,,,,,,,,,,,,,,,,,,,,,
Moroccan_cookies_for_school,P_7290017046051,Pastries_and_cakes_from_the_bakery,0.0,0.0,0.0,0.0,485.0,7.7,67.2,24.4,290.0,...,0,0,0,0,0,0,0,0,0,0.0
Chickpea_Chips_Chips_L._Gluten_Free,P_764218652672,Gluten_free,0.0,0.0,3.5,0.0,465.0,7.9,68.0,18,0.0,...,0,0,0,0,0,0,0,0,0,0.0
Yopla_on_top_crackles,P_7290110326210,Milk_and_eggs,0.0,0.0,0.0,0.0,108.0,4.5,12.8,4.3,58.0,...,0,0,0,0,0,0,0,0,0,0.0
Chocolate_spread_for_GHS,P_8410342003508,Diet_and_sugar_free,0.0,0.0,0.0,464.0,5.0,51.0,34.0,28,0.0,...,0,0,0,0,0,0,0,0,0,NaN
Chocolate_Mercy_Nut_Hazelnut_+_Almond,P_4014400914252,snacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Candy_Goldvers,P_9002975393843,snacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
Slim_chocolate_bitters,P_8423207206495,snacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
Market.Milka_Oreo_Sandwich,P_7622210826053,snacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0,0,0,0,0,0,0,0,0.0


In [9]:
products_notscaled = prod.drop("category_name", axis=1)
category = prod["category_name"].copy()

In [10]:
for col in products_notscaled.columns:  
    products_notscaled[col] = pd.to_numeric(products_notscaled[col], errors='coerce')
products_notscaled = products_notscaled.fillna(0)
sts_dropped_collumns = np.transpose(products_notscaled.loc[:, (products_notscaled == 0).all(axis=0)]).index
products_notscaled = products_notscaled.loc[:, (products_notscaled != 0).any(axis=0)]
nulls = products_notscaled.isnull().sum()

In [11]:
sts_dropped_collumns

Index(['DHA_+_EPA', 'From_sugars_to_organic_acids', 'wetness', 'ash',
       'Linoleic_fatty_acid', 'Alpha_linolenic_acid', 'humidity', 'chromium',
       'Chlorophyll'],
      dtype='object')

In [12]:
nulls = products_notscaled.isnull().sum()
nulls

Vitamin_B1       0
Vitamin_B6       0
Dietary_fiber    0
Folic_acid       0
energy           0
                ..
Unsaturated      0
Acidic_acid      0
Linoleic_acid    0
DHA              0
EPA              0
Length: 88, dtype: int64

In [13]:
#in order to reduce background noise, we decided to drop the columns in which there are less than 10 products_notscaleducts with this feature
products_notscaled = products_notscaled.reset_index()
arr = np.array([])
col_droped = np.array([])
for col in products_notscaled.columns:
    if products_notscaled[col].astype(bool).sum(axis=0)<20: 
        arr = np.append(arr, products_notscaled[col].to_numpy().nonzero())
        col_droped = np.append(col_droped, col)
print(arr)
arr = set(arr) #all number of rows without duplicates
for i in arr:
    products_notscaled = products_notscaled.drop(i)
products_notscaled = products_notscaled.loc[:, (products_notscaled != 0).any(axis=0)] #deletes all columns with all values set to zero
products_notscaled = products_notscaled.set_index(['product_name', 'code'])

[ 196. 1305. 2003. 2592. 2921. 3298. 3439. 3826. 4364. 4985. 5035. 5503.
 5759. 6057. 6818. 7003. 7028. 7044.  289.  960. 1558. 1570. 1828. 2176.
 2719. 3132. 3669. 3719. 4194. 5289. 5601. 6263. 6892.   99.  448. 2790.
 3202. 4700. 5815. 6413. 6430.  326.  331.  630.  786. 1638. 2647. 2839.
 3870. 4451. 4700. 4703. 4845. 5202. 6081. 6465. 6747. 7272.  392. 1356.
 4413. 2341. 2637. 2952. 3083. 4638. 4653. 5573. 5957. 6137. 6241. 6576.
 7404. 1869. 2341. 2637. 2765. 3083. 4548. 4653. 4757. 4926. 5573. 5957.
 6094. 6137. 7404. 1548. 2921. 3439. 4751. 5168.  883. 1287. 1410. 2114.
 2224. 2318. 2526. 2755. 3046. 3139. 3595. 3892. 4636. 5428. 6307. 6805.
 7051.  902. 1305. 3784. 5103. 1111. 4926. 2501. 3780.  180.  868. 2647.
 4253. 4375. 4700. 4845. 6747.  145. 1214. 1786. 2268. 2587. 3956. 4095.
 4413. 7132.  512.  675. 1192. 2973. 4437. 4901.  686. 1314. 4314. 7016.
 7027. 1034. 1392. 1800. 3227. 4375. 4700.   99.  448. 1483. 1558. 2790.
 3006. 3202. 3669. 3719. 5268. 5601. 5815. 6053. 64

In [14]:
category = category.reset_index()
for i in arr:
    category = category.drop(i)
category = category.set_index(['product_name', 'code'])    

In [23]:
ordinal_encoder = OrdinalEncoder()
category_encoded = ordinal_encoder.fit_transform(category)
category_encoded[:10]
categories = ordinal_encoder.categories_
print(categories[0])
enc = OneHotEncoder()
categoryarr = enc.fit_transform(category)
category1 = pd.get_dummies(categoryarr.getnnz(axis=0))
data = pd.get_dummies(categoryarr.getnnz(),prefix=categories, columns = categories, drop_first=True)

#print(categoryarr.shape)
#category1 = pd.DataFrame(data=categoryarr)
#x=0
#for cater in categories[0]:
    #category1[cater] = category1[28-x]
    #x=x+1
#category1 = category1.loc[:,'Babies_and_children':'vegetables']

['Babies_and_children' 'Baking_and_canning_cooking' 'Beef_and_lamb'
 'Breads' 'Cereals' 'Chicken_and_turkey_products' 'Clearance_of_inventory'
 'Cooking' 'Cooking_and_baking_products' 'Diet_and_sugar_free' 'Fish'
 'For_baby_-_food' 'For_those_who_avoid_gluten' 'From_the_freezer'
 'Gluten_free' 'Health_in_the_refrigerator' 'House_cleaning_and_one-time'
 'Milk_and_eggs' 'Nuts_and_dried_fruits' 'Organic_fruits_and_vegetables'
 'Pastries_and_cakes_from_the_bakery' 'Refrigerated_food'
 'Refrigerator_products_and_eggs' 'Vitamins_and_supplements'
 'Without_added_sugar' 'bread' 'fruits' 'snacks' 'vegetables']


TypeError: object of type 'int' has no len()

In [18]:
category1

,1,4,5,6,9,15,26,31,48,62,...,182,188,195,232,256,280,939,1036,1452,1533
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ordinal_encoder = OrdinalEncoder()
category_encoded = ordinal_encoder.fit_transform(category)
category_encoded[:10]
categories = ordinal_encoder.categories_
print(categories[0])
enc = OneHotEncoder()
categoryarr = enc.fit_transform(category)
data = pd.get_dummies(categoryarr.getnnz())

In [ ]:
data

In [ ]:
col_droped

In [ ]:
products_notscaled

In [ ]:
scaler = StandardScaler()
scaler.fit(products_notscaled)
productsarr = scaler.transform(products_notscaled) #scaler returns numpy array
products = pd.DataFrame(productsarr, index=products_notscaled.index, columns=products_notscaled.columns)

In [ ]:
def merge_two_columns(col1, col2, dataframe, index_col):
    if col1 in dataframe.columns and col2 in dataframe.columns:
        dataframe = dataframe.reset_index()
        row = 0
        for prod in dataframe[col1]:
            if (prod==0):
                if (dataframe.at[row , col2]!=0):
                    dataframe.at[row, col1] =  dataframe.at[row ,col2]  
            row = row+1
        dataframe = dataframe.drop([col2], axis = 1)
        dataframe = dataframe.set_index(index_col, drop=True)
        return dataframe
    else:
        return dataframe

In [ ]:
products.columns

In [ ]:
products = merge_two_columns('Vitamin_A', 'Vitamin_A_is_worth_retinol', products, ['product_name', 'code'])
products = merge_two_columns('Dietary_fiber', 'Dietary_fiber.1', products, ['product_name', 'code'])
products = merge_two_columns('potassium', 'K_Potassium', products, ['product_name', 'code'])
products = merge_two_columns('Vitamin_E.', 'Vitamin_E_equals_alpha_tocopherol', products, ['product_name', 'code'])
products = merge_two_columns('Vitamin_D.', 'Vitamin_D_Calciprolol', products, ['product_name', 'code'])
products = merge_two_columns('Vitamin_B3', 'Niacin', products, ['product_name', 'code'])
products = merge_two_columns('Vitamin_B3', 'Vitamin_B3_Nicotinamide_Equilibrium', products, ['product_name', 'code'])
products = merge_two_columns('Vitamin_B3', 'Nicotinamide', products, ['product_name', 'code'])
products = merge_two_columns('sodium', 'chloride', products, ['product_name', 'code'])
products['Partof_vitaminB'] = products['Vitamin_B6']
products = products.drop(['Vitamin_B6','Vitamin_B1','Vitamin_B2','Vitamin_B12','Folic_acid'], axis=1)
products = products.drop(['Carbohydrate_sugars'], axis=1)
products = products.drop(['energy'], axis=1)
products['6_omega_fatty_acids'] = products['ARA_fatty_acid']
products = merge_two_columns('6_omega_fatty_acids', 'ARA_fatty_acid', products, ['product_name', 'code'])
products = merge_two_columns('6_omega_fatty_acids', 'Linoleic_fatty_acid', products, ['product_name', 'code'])
products = merge_two_columns('3_omega_fatty_acids', 'DHA_fatty_acid', products, ['product_name', 'code'])
products = merge_two_columns('3_omega_fatty_acids', 'Alpha_linolenic_acid', products, ['product_name', 'code'])

In [ ]:
k_range = list(range(1, 31))
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(products, category.values.ravel())
    cat_pred = knn.predict(products)
    scores.append(metrics.accuracy_score(category, cat_pred))
    
plt.plot(k_range, scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

In [ ]:
k_range = list(range(1, 31))
k_mean_scores = []
k_std_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, products, category.values.ravel(), cv=10, scoring='accuracy')
    k_mean_scores.append(scores.mean())
    k_std_scores.append(scores.std())

d = {'k_mean': k_mean_scores, 'k_std': k_std_scores}
Ks = pd.DataFrame(data = d, index=k_range)